In [1]:
import os
from verification_system import VerificationSystem
import matplotlib.pyplot as plt

# Eksperyment bez fine-tuningu

In [2]:
verification_system = VerificationSystem(
    database_path=os.path.join("data", "database"),
)

lum_fine_authorized_users_linear_0.5
24-04-07 12:57:01 - Searching data/database/incoming_users/authorized_users/25/010802.jpg in 4409 length datastore
24-04-07 12:57:01 - find function duration 1.1351170539855957 seconds
authorized_users
24-04-07 12:57:01 - Searching data/database/incoming_users/authorized_users/25/010802.jpg in 2206 length datastore
24-04-07 12:57:01 - find function duration 0.2187032699584961 seconds
gauss_fine_authorized_users_psnr20
24-04-07 12:57:01 - Found 4386 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations:   1%|▏         | 58/4386 [00:08<10:23,  6.94it/s]


KeyboardInterrupt: 

In [ ]:
exp = os.listdir("data/database/incoming_users")
exp_lum = list(filter(lambda x: "lum" in x, exp))
exp_psnr = list(filter(lambda x: "psnr" in x, exp))

In [ ]:
lumsplit = lambda x: x.split("_")[0] + ' ' + x.split("_")[-2] + ': ' + x.split("_")[-1]
psnrsplit = lambda x: x.split("_")[0] + ' ' + x.split("_")[-1]

In [ ]:
for in_dir in exp_lum:
    path = os.path.join("data/database/incoming_users", in_dir)
    expkey = lumsplit(in_dir)
    try:
        df_multi = verification_system.verify_multiple_users(path)
        res_lum[expkey] = verification_system.calculate_access_granted_rate(df_users=df_multi)
    except Exception as e:
        print(f"Error: {e}")
        res_lum[expkey] = -1
        continue
    

In [ ]:
try:
    plt.bar(res_lum.keys(), res_lum.values())
    plt.xticks(rotation=90)
    plt.title("Access granted rate for different luminance levels")
    plt.xlabel("Luminance level")
    plt.ylabel("Access granted rate")
    plt.show()
except Exception as e:
    print(f"Error: {e}")
finally:
    res_lum = {k: v for k, v in res_lum.items() if v is not None}
    plt.bar(res_lum.keys(), res_lum.values())
    plt.xticks(rotation=90)
    plt.title("Access granted rate for different luminance levels")
    plt.xlabel("Luminance level")
    plt.ylabel("Access granted rate")
    plt.show()

In [ ]:
for in_dir in exp_psnr:
    path = os.path.join("data/database/incoming_users", in_dir)
    expkey = psnrsplit(in_dir)
    print('Experimenting with: ', expkey)
    try:
        df_multi = verification_system.verify_multiple_users(path)
        res_psnr[expkey] = verification_system.calculate_access_granted_rate(df_users=df_multi)
    except Exception as e:
        print(f"Error: {e}")
        res_psnr[expkey] = -1
        continue


In [ ]:
try:
    plt.bar(res_psnr.keys(), res_psnr.values())
    plt.xticks(rotation=90)
    plt.title("Access granted rate for different PSNR levels")
    plt.xlabel("PSNR level")
    plt.ylabel("Access granted rate")
    plt.show()
except Exception as e:
    print(f"Error: {e}")
finally:
    res_psnr = {k: v for k, v in res_psnr.items() if v is not None}
    plt.bar(res_psnr.keys(), res_psnr.values())
    plt.xticks(rotation=90)
    plt.title("Access granted rate for different PSNR levels")
    plt.xlabel("PSNR level")
    plt.ylabel("Access granted rate")
    plt.show()

# Eksperyment z fine-tuningu

In [ ]:
# najpierw w utils/exp_database stworzyć foldery na bazy danych z takimi parametrami jakie chcemy fintunować